#同時方程式体系
『Rによる計量経済学』第10章「同時方程式体系」をPythonで実行する。  
テキスト付属データセット(「k1001.csv」等)については出版社サイトよりダウンロードしてください。  
また、以下の説明は本書の一部を要約したものですので、より詳しい説明は本書を参照してください。   

###例題10.1
次のような供給関数と需要関数を推定する。  
$Q_{t} = \alpha_{0} + \alpha_{1} P_{t} + \alpha_{2} E_{t} + u_{t}$  
$Q_{t} = \beta_{0} + \beta_{1} P_{t} + \beta_{2} A_{t} + v_{t}$  
ただし、$Q_{t}$ は数量、$P_{t}$ は価格、$E_{t}$ は供給関数シフト要因、$A_{t}$ は需要関数シフト要因とする。

In [1]:
%matplotlib inline

In [2]:
# -*- coding:utf-8 -*-
from __future__ import print_function
import numpy as np
import pandas as pd
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
import matplotlib.pyplot as plt

In [3]:
# データ読み込み
data = pd.read_csv('example/k1001.csv')

In [4]:
# 式1説明変数設定
X1 = data[['P', 'E']].as_matrix().reshape(-1, 2)
X1 = sm.add_constant(X1)
# 式2説明変数設定
X2 = data[['P', 'A']].as_matrix().reshape(-1, 2)
X2 = sm.add_constant(X2)

In [5]:
# 被説明変数設定
Y = data[['Q']].as_matrix().reshape(-1)

In [6]:
# OLSの実行(Ordinary Least Squares: 最小二乗法)
model1 = sm.OLS(Y, X1)
model2 = sm.OLS(Y, X2)
result1 = model1.fit()
result2 = model2.fit()

In [7]:
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.529
Model:                            OLS   Adj. R-squared:                  0.474
Method:                 Least Squares   F-statistic:                     9.548
Date:                Sun, 19 Jul 2015   Prob (F-statistic):            0.00166
Time:                        02:56:37   Log-Likelihood:                -22.259
No. Observations:                  20   AIC:                             50.52
Df Residuals:                      17   BIC:                             53.51
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          4.8581      0.695      6.988      0.0

In [8]:
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.627
Model:                            OLS   Adj. R-squared:                  0.583
Method:                 Least Squares   F-statistic:                     14.30
Date:                Sun, 19 Jul 2015   Prob (F-statistic):           0.000227
Time:                        02:56:37   Log-Likelihood:                -19.920
No. Observations:                  20   AIC:                             45.84
Df Residuals:                      17   BIC:                             48.83
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         16.6747      1.905      8.752      0.0

この結果から古典的最小二乗法による推定式をまとめると、  
[供給関数]   
$\hat Q_{i} = 4.8581 + 1.5094 P_{i} - 1.5202 E_{i} $  
[需要関数]  
$\hat Q_{i} = 16.6747 - 0.9088 P_{i} - 1.0369 A_{i}$  
となる。  

しかし、説明変数Pと誤差の間に関係があるため、同時方程式バイアスが生じてしまいます。  

そこで、以下では同時方程式体系の推定法として代表的な**二段階最小二乗法**を用いて推定し直します。

In [9]:
# 外生変数設定
inst = data[[ 'A', 'E']].as_matrix()
inst = sm.add_constant(inst)

In [10]:
# 2SLSの実行(Two Stage Least Squares: 二段階最小二乗法)
model1 = IV2SLS(Y, X1, inst)
model2 = IV2SLS(Y, X2, inst)
result1 = model1.fit()
result2 = model2.fit()

In [11]:
print(result1.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.434
Model:                         IV2SLS   Adj. R-squared:                  0.367
Method:                     Two Stage   F-statistic:                     10.66
                        Least Squares   Prob (F-statistic):            0.00100
Date:                Sun, 19 Jul 2015                                         
Time:                        02:56:37                                         
No. Observations:                  20                                         
Df Residuals:                      17                                         
Df Model:                           2                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const          3.7867      0.889      4.259      0.0

In [12]:
print(result2.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                      y   R-squared:                       0.618
Model:                         IV2SLS   Adj. R-squared:                  0.573
Method:                     Two Stage   F-statistic:                     15.80
                        Least Squares   Prob (F-statistic):           0.000133
Date:                Sun, 19 Jul 2015                                         
Time:                        02:56:37                                         
No. Observations:                  20                                         
Df Residuals:                      17                                         
Df Model:                           2                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
const         17.8558      2.023      8.828      0.0

この結果から二段階最小二乗法による推定式をまとめると、  
[供給関数]   
$\hat Q_{i} = 3.7867  + 2.2119  P_{i} - 2.1531 E_{i} $  
[需要関数]  
$\hat Q_{i} = 17.8558 - 1.0249 P_{i} - 1.1484 A_{i}$  
となる。  